# Notebook to Implement Model Training - LGBM

---

### 1) Setup

In [283]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from scipy import stats
from scipy.stats import wilcoxon

from sklearn.metrics import balanced_accuracy_score, make_scorer
from sklearn.model_selection import RepeatedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

import optuna
import lightgbm as lgb

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

In [2]:
SEED = 42

In [3]:
TRAIN_CLINICAL_FILENAME = "train_set.csv"
TEST_CLINICAL_FILENAME = "test_set.csv"

---

### 2) Read and Preprocess Data

In [4]:
train = pd.read_csv(TRAIN_CLINICAL_FILENAME, sep=",", index_col="ID")

In [5]:
train.shape

(132, 636)

In [6]:
train.head()

,Freq.1198.05707939985,Freq.1204.07148226988,Freq.1211.25531677913,Freq.1217.79534957812,Freq.1223.24180676615,Freq.1234.42874922858,Freq.1239.04873179697,Freq.1243.98934968702,Freq.1249.10239635402,Freq.1254.7097579948,...,Freq.8943.76551923189,Freq.9058.85825530971,Freq.9098.58510797401,Freq.9437.74469644083,Freq.9593.90405666006,Freq.9799.842201746,Freq.10432.4853106264,Freq.11006.9514551194,Freq.11161.31855876,Group
ID,,,,,,,,,,,,,,,,,,,,,
1,0.000319,0.000416,0.000077,4.034140e-05,0.000089,0.000282,0.000004,0.000130,0.000288,0.000000e+00,...,0.000017,0.000005,0.000020,0.000029,0.000037,0.000024,0.000016,0.000030,0.000030,MILD
10,0.000000,0.000000,0.000021,1.064014e-03,0.000113,0.001591,0.016310,0.002330,0.000476,6.074661e-04,...,0.000040,0.000010,0.000001,0.000005,0.000002,0.000052,0.000003,0.000155,0.000001,MILD
100,0.000199,0.000006,0.000013,5.089552e-08,0.000087,0.000543,0.000231,0.000068,0.000018,1.286428e-04,...,0.000009,0.000044,0.000012,0.000067,0.000207,0.000020,0.000027,0.000100,0.000100,MILD
101,0.000196,0.000244,0.000069,4.874671e-05,0.000125,0.000353,0.000029,0.000125,0.000120,4.988436e-07,...,0.000072,0.000051,0.000013,0.000061,0.000040,0.000008,0.000003,0.000055,0.000029,MILD
105,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000027,0.000031,0.000014,0.000017,0.000028,0.000009,0.000012,0.000026,0.000017,MILD


In [7]:
##### Preprocessing

train_mod = train.copy()
# Imputation using mean method
#train_mod = train_mod.replace(0,np.nan)
#train_mod_imp = train_mod.transform(lambda x: x.fillna(x.mean()))

In [8]:
train_mod.shape

(132, 636)

In [9]:
train_mod.head()

,Freq.1198.05707939985,Freq.1204.07148226988,Freq.1211.25531677913,Freq.1217.79534957812,Freq.1223.24180676615,Freq.1234.42874922858,Freq.1239.04873179697,Freq.1243.98934968702,Freq.1249.10239635402,Freq.1254.7097579948,...,Freq.8943.76551923189,Freq.9058.85825530971,Freq.9098.58510797401,Freq.9437.74469644083,Freq.9593.90405666006,Freq.9799.842201746,Freq.10432.4853106264,Freq.11006.9514551194,Freq.11161.31855876,Group
ID,,,,,,,,,,,,,,,,,,,,,
1,0.000319,0.000416,0.000077,4.034140e-05,0.000089,0.000282,0.000004,0.000130,0.000288,0.000000e+00,...,0.000017,0.000005,0.000020,0.000029,0.000037,0.000024,0.000016,0.000030,0.000030,MILD
10,0.000000,0.000000,0.000021,1.064014e-03,0.000113,0.001591,0.016310,0.002330,0.000476,6.074661e-04,...,0.000040,0.000010,0.000001,0.000005,0.000002,0.000052,0.000003,0.000155,0.000001,MILD
100,0.000199,0.000006,0.000013,5.089552e-08,0.000087,0.000543,0.000231,0.000068,0.000018,1.286428e-04,...,0.000009,0.000044,0.000012,0.000067,0.000207,0.000020,0.000027,0.000100,0.000100,MILD
101,0.000196,0.000244,0.000069,4.874671e-05,0.000125,0.000353,0.000029,0.000125,0.000120,4.988436e-07,...,0.000072,0.000051,0.000013,0.000061,0.000040,0.000008,0.000003,0.000055,0.000029,MILD
105,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000027,0.000031,0.000014,0.000017,0.000028,0.000009,0.000012,0.000026,0.000017,MILD


In [84]:
test = pd.read_csv(TEST_CLINICAL_FILENAME, sep=",", index_col="ID")
test_mod = test.copy()

In [85]:
X_test, y_test = test_mod.drop("Group", axis=1), test_mod["Group"]

---

### 3) Baseline Model Training and CV

In [10]:
# Define Classifier (or pipeline)
clf = lgb.LGBMClassifier(random_state=SEED)

In [12]:
# Get Features and Target
X, y = train_mod.drop("Group", axis=1), train_mod["Group"]

In [13]:
# Defining RepeatedKFold Cross Validator
rkf = RepeatedKFold(n_splits=5, n_repeats=20, random_state=SEED)

In [39]:
# Define metric scorer
metric_scorer = make_scorer(balanced_accuracy_score)
metric_scorer

make_scorer(balanced_accuracy_score)

In [15]:
# Cross validate model
scores = cross_val_score(clf, X, y, scoring=metric_scorer, cv=rkf, n_jobs=-1)

In [16]:
# Score from each CV Iteration
scores

array([0.82236842, 0.75      , 0.86842105, 0.73076923, 0.85119048,
       0.875     , 0.875     , 0.76190476, 0.7375    , 0.88888889,
       0.82954545, 0.79117647, 0.72556391, 0.72916667, 0.85625   ,
       0.83333333, 0.85      , 0.85714286, 0.80065359, 0.69281046,
       0.63333333, 0.88486842, 0.70606061, 0.8       , 0.81875   ,
       0.80833333, 0.88736264, 0.59722222, 0.69281046, 0.85947712,
       0.69444444, 0.83235294, 0.77819549, 0.78125   , 0.69281046,
       0.69602273, 0.78618421, 0.72619048, 0.56666667, 0.76785714,
       0.75      , 0.75      , 0.83030303, 0.83030303, 0.88194444,
       0.78021978, 0.90909091, 0.80392157, 0.7875    , 0.82330827,
       0.8       , 0.64166667, 0.7875    , 0.74509804, 0.81875   ,
       0.83238636, 0.75      , 0.70138889, 0.80357143, 0.75      ,
       0.72058824, 0.69444444, 0.80952381, 0.84722222, 0.64285714,
       0.68333333, 0.8       , 0.88194444, 0.72916667, 0.83030303,
       0.78693182, 0.79117647, 0.6875    , 0.91176471, 0.76363

In [17]:
# Mean Metric Value
np.mean(scores)

0.7818376105748396

# Feature reduction

In [71]:
train_mod_stats = train_mod.drop('Group', axis = 1)


In [72]:
df = train_mod.copy()
peaks_list = []
pval_list = []
df2 = pd.melt(df, id_vars = ["Group"])
mild = df.where(df.Group == "MILD").dropna()
sev = df.where(df.Group == "SEVERE").dropna()
for i in range(1,(len(df.columns)-1)):
    s, p1 = stats.mannwhitneyu(mild.iloc[:,i], sev.iloc[:,i], alternative = 'two-sided')
    if p1 <= 0.05:
        pval_list.append(p1)
        peaks_list.append(df.columns[i])
peaks_list.append("Group")

In [87]:
df_filtered = df.drop(columns=[col for col in df if col not in peaks_list])
df_test_filtered = test.drop(columns=[col for col in test if col not in peaks_list])

In [89]:
df_test_filtered.shape

(64, 150)

---

In [97]:
# Get Features and Target
X, y = df_filtered.drop("Group", axis=1), df_filtered["Group"]
X_test, y_test = df_test_filtered.drop("Group", axis=1), df_test_filtered["Group"]
# Cross validate model
scores = cross_val_score(clf, X, y, scoring=metric_scorer, cv=rkf, n_jobs=-1)
# Score from each CV Iteration
scores

array([0.79605263, 0.88888889, 0.82330827, 0.76923077, 0.96428571,
       0.88333333, 0.75986842, 0.80357143, 0.7375    , 0.88888889,
       0.79829545, 0.82058824, 0.7518797 , 0.75694444, 0.86875   ,
       0.88888889, 0.9       , 0.81547619, 0.82679739, 0.71568627,
       0.70833333, 0.88486842, 0.71818182, 0.8       , 0.81875   ,
       0.73333333, 0.88736264, 0.60416667, 0.71895425, 0.83006536,
       0.80555556, 0.83235294, 0.80451128, 0.875     , 0.72222222,
       0.78693182, 0.8125    , 0.79761905, 0.625     , 0.85119048,
       0.75      , 0.8       , 0.73939394, 0.86363636, 0.88194444,
       0.8543956 , 0.87784091, 0.83006536, 0.86875   , 0.82330827,
       0.9       , 0.725     , 0.80625   , 0.71568627, 0.8875    ,
       0.95454545, 0.8       , 0.70138889, 0.88095238, 0.7       ,
       0.72058824, 0.75      , 0.80952381, 0.78472222, 0.68452381,
       0.725     , 0.76666667, 0.81944444, 0.79166667, 0.84242424,
       0.86363636, 0.79117647, 0.75625   , 0.94117647, 0.76363

In [92]:
# Mean Metric Value
np.mean(scores)

0.8039530045547848

### 4) Experiments

##### 4.1) Hyper Parameter Optimization with Optuna

In [128]:
# Define objectiva funtion to maximize metric
def objective(trial):
    
    # Hyper Parameter Grid
    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "n_estimators": trial.suggest_int("max_depth", 100, 1000)
    }
    # Create model with param trial
    clf = lgb.LGBMClassifier(random_state=SEED, **param)
    # Get CV Metric we want to maximize
    balanced_accuracy = np.mean(cross_val_score(clf, X, y, scoring=metric_scorer, cv=rkf, n_jobs=-1))
    
    return balanced_accuracy

In [129]:
study = optuna.create_study(direction="maximize")

[I 2022-06-21 15:45:39,282] A new study created in memory with name: no-name-e6416ef2-db4b-43fe-88d2-f515ea06571a


In [130]:
study.optimize(objective, n_trials=500)

[I 2022-06-21 15:46:12,124] Trial 0 finished with value: 0.5 and parameters: {'lambda_l1': 9.94178690130804e-07, 'lambda_l2': 6.396882700818826e-07, 'num_leaves': 233, 'feature_fraction': 0.6538636920379493, 'bagging_fraction': 0.4738351942510842, 'bagging_freq': 5, 'min_child_samples': 34, 'max_depth': 822}. Best is trial 0 with value: 0.5.
[I 2022-06-21 15:46:15,559] Trial 1 finished with value: 0.8521005587120137 and parameters: {'lambda_l1': 0.013322054229437065, 'lambda_l2': 0.5502889088762963, 'num_leaves': 235, 'feature_fraction': 0.9108649897363823, 'bagging_fraction': 0.8063519516448048, 'bagging_freq': 5, 'min_child_samples': 44, 'max_depth': 362}. Best is trial 1 with value: 0.8521005587120137.
[I 2022-06-21 15:46:17,689] Trial 2 finished with value: 0.5 and parameters: {'lambda_l1': 0.03688252904299475, 'lambda_l2': 2.8541859675529306e-06, 'num_leaves': 225, 'feature_fraction': 0.8014690743573696, 'bagging_fraction': 0.5733983875978387, 'bagging_freq': 2, 'min_child_samples

[I 2022-06-21 15:47:30,200] Trial 22 finished with value: 0.7176173893830626 and parameters: {'lambda_l1': 1.250110199888723e-07, 'lambda_l2': 0.005649106517137179, 'num_leaves': 142, 'feature_fraction': 0.5441127177991123, 'bagging_fraction': 0.5363743250007063, 'bagging_freq': 3, 'min_child_samples': 34, 'max_depth': 486}. Best is trial 16 with value: 0.8811761166679898.
[I 2022-06-21 15:47:34,201] Trial 23 finished with value: 0.8714963405402648 and parameters: {'lambda_l1': 2.4073326244501874e-06, 'lambda_l2': 0.10512963899232872, 'num_leaves': 99, 'feature_fraction': 0.4786620653582086, 'bagging_fraction': 0.4024229090219423, 'bagging_freq': 4, 'min_child_samples': 17, 'max_depth': 526}. Best is trial 16 with value: 0.8811761166679898.
[I 2022-06-21 15:47:43,234] Trial 24 finished with value: 0.8511374290484198 and parameters: {'lambda_l1': 7.94983213297775e-05, 'lambda_l2': 0.002349543795169676, 'num_leaves': 187, 'feature_fraction': 0.5762394303001239, 'bagging_fraction': 0.6251

[I 2022-06-21 15:49:14,911] Trial 44 finished with value: 0.8681508221096069 and parameters: {'lambda_l1': 6.080460751761331e-07, 'lambda_l2': 0.0001878636759265941, 'num_leaves': 186, 'feature_fraction': 0.4050047261452406, 'bagging_fraction': 0.647933176578071, 'bagging_freq': 5, 'min_child_samples': 32, 'max_depth': 509}. Best is trial 16 with value: 0.8811761166679898.
[I 2022-06-21 15:49:16,793] Trial 45 finished with value: 0.5 and parameters: {'lambda_l1': 2.0458701382487298e-07, 'lambda_l2': 2.3280524436107897e-05, 'num_leaves': 232, 'feature_fraction': 0.5948706876621798, 'bagging_fraction': 0.42741963824656126, 'bagging_freq': 3, 'min_child_samples': 45, 'max_depth': 566}. Best is trial 16 with value: 0.8811761166679898.
[I 2022-06-21 15:49:20,575] Trial 46 finished with value: 0.8665419422338927 and parameters: {'lambda_l1': 2.15048050488611e-06, 'lambda_l2': 0.00698113438937916, 'num_leaves': 126, 'feature_fraction': 0.7425199592895938, 'bagging_fraction': 0.585042196581778

[I 2022-06-21 15:51:04,979] Trial 66 finished with value: 0.5 and parameters: {'lambda_l1': 2.7582919511685486e-06, 'lambda_l2': 0.10198250997548158, 'num_leaves': 180, 'feature_fraction': 0.42428969610501416, 'bagging_fraction': 0.4012261999929807, 'bagging_freq': 4, 'min_child_samples': 34, 'max_depth': 532}. Best is trial 16 with value: 0.8811761166679898.
[I 2022-06-21 15:51:09,102] Trial 67 finished with value: 0.8687060445574377 and parameters: {'lambda_l1': 7.349417058410406e-05, 'lambda_l2': 0.0006164976182081101, 'num_leaves': 146, 'feature_fraction': 0.48707408304042954, 'bagging_fraction': 0.6224373002862733, 'bagging_freq': 4, 'min_child_samples': 20, 'max_depth': 405}. Best is trial 16 with value: 0.8811761166679898.
[I 2022-06-21 15:51:15,061] Trial 68 finished with value: 0.8680702605143393 and parameters: {'lambda_l1': 1.1971148788182363e-06, 'lambda_l2': 2.0151149702599686e-07, 'num_leaves': 138, 'feature_fraction': 0.5433981203547671, 'bagging_fraction': 0.59341654637

[I 2022-06-21 15:52:59,937] Trial 88 finished with value: 0.8799582241588818 and parameters: {'lambda_l1': 2.0197136013670465e-06, 'lambda_l2': 0.00022569588894087297, 'num_leaves': 190, 'feature_fraction': 0.4517570060682192, 'bagging_fraction': 0.5611025796797239, 'bagging_freq': 1, 'min_child_samples': 19, 'max_depth': 455}. Best is trial 16 with value: 0.8811761166679898.
[I 2022-06-21 15:53:04,392] Trial 89 finished with value: 0.8749241924226445 and parameters: {'lambda_l1': 6.174449190177766e-06, 'lambda_l2': 0.007360102391987431, 'num_leaves': 148, 'feature_fraction': 0.5686208663918747, 'bagging_fraction': 0.5498489044108533, 'bagging_freq': 1, 'min_child_samples': 19, 'max_depth': 445}. Best is trial 16 with value: 0.8811761166679898.
[I 2022-06-21 15:53:08,853] Trial 90 finished with value: 0.8623355753723402 and parameters: {'lambda_l1': 2.5772732476567496e-07, 'lambda_l2': 0.0025676598939141527, 'num_leaves': 107, 'feature_fraction': 0.5056408287854305, 'bagging_fraction':

...

[I 2022-06-21 16:24:26,009] Trial 407 finished with value: 0.8813093860903457 and parameters: {'lambda_l1': 0.0001559705119274011, 'lambda_l2': 2.912580296393889e-05, 'num_leaves': 49, 'feature_fraction': 0.43974907846349265, 'bagging_fraction': 0.47177724519081876, 'bagging_freq': 1, 'min_child_samples': 21, 'max_depth': 852}. Best is trial 205 with value: 0.8875892335648526.
[I 2022-06-21 16:24:32,796] Trial 408 finished with value: 0.8792980045625169 and parameters: {'lambda_l1': 0.00044716872487764734, 'lambda_l2': 0.0005414503718799589, 'num_leaves': 69, 'feature_fraction': 0.42329468521276276, 'bagging_fraction': 0.4609659155718737, 'bagging_freq': 1, 'min_child_samples': 12, 'max_depth': 825}. Best is trial 205 with value: 0.8875892335648526.
[I 2022-06-21 16:24:39,460] Trial 409 finished with value: 0.8780072399865355 and parameters: {'lambda_l1': 0.0002823710053549648, 'lambda_l2': 0.00018639433754600988, 'num_leaves': 33, 'feature_fraction': 0.43142987612145545, 'bagging_frac

[I 2022-06-21 16:26:33,766] Trial 428 finished with value: 0.8760069154701506 and parameters: {'lambda_l1': 0.00018616897031614155, 'lambda_l2': 0.0006218108469178078, 'num_leaves': 31, 'feature_fraction': 0.4226192928663138, 'bagging_fraction': 0.4138852371770448, 'bagging_freq': 1, 'min_child_samples': 21, 'max_depth': 970}. Best is trial 205 with value: 0.8875892335648526.
[I 2022-06-21 16:26:45,870] Trial 429 finished with value: 0.865929099495276 and parameters: {'lambda_l1': 0.0007777179651516396, 'lambda_l2': 6.019226485390526e-05, 'num_leaves': 57, 'feature_fraction': 0.40902717870025423, 'bagging_fraction': 0.5001134908464107, 'bagging_freq': 7, 'min_child_samples': 19, 'max_depth': 921}. Best is trial 205 with value: 0.8875892335648526.
[I 2022-06-21 16:26:54,027] Trial 430 finished with value: 0.8804986571735798 and parameters: {'lambda_l1': 0.0003703804906812325, 'lambda_l2': 3.847506279793461e-05, 'num_leaves': 69, 'feature_fraction': 0.4305538433819509, 'bagging_fraction'

[I 2022-06-21 16:29:30,885] Trial 450 finished with value: 0.8829844590394127 and parameters: {'lambda_l1': 5.553263541382837e-05, 'lambda_l2': 0.0010860916988087734, 'num_leaves': 22, 'feature_fraction': 0.4142207269556086, 'bagging_fraction': 0.45149112661633495, 'bagging_freq': 1, 'min_child_samples': 14, 'max_depth': 899}. Best is trial 205 with value: 0.8875892335648526.
[I 2022-06-21 16:29:41,754] Trial 451 finished with value: 0.8835561930759297 and parameters: {'lambda_l1': 0.0005025289137102649, 'lambda_l2': 0.0034995328677811756, 'num_leaves': 42, 'feature_fraction': 0.44554320025511407, 'bagging_fraction': 0.46831574238564416, 'bagging_freq': 1, 'min_child_samples': 17, 'max_depth': 856}. Best is trial 205 with value: 0.8875892335648526.
[I 2022-06-21 16:29:49,972] Trial 452 finished with value: 0.877560623007024 and parameters: {'lambda_l1': 0.0012305072875005484, 'lambda_l2': 0.0008489118221944081, 'num_leaves': 47, 'feature_fraction': 0.6741309635539432, 'bagging_fraction

[I 2022-06-21 16:32:18,091] Trial 471 finished with value: 0.8610537495485251 and parameters: {'lambda_l1': 0.0003701786403513371, 'lambda_l2': 0.00018089292695456838, 'num_leaves': 43, 'feature_fraction': 0.47126875592502293, 'bagging_fraction': 0.4587047884161032, 'bagging_freq': 1, 'min_child_samples': 9, 'max_depth': 880}. Best is trial 205 with value: 0.8875892335648526.
[I 2022-06-21 16:32:24,764] Trial 472 finished with value: 0.8768835244562798 and parameters: {'lambda_l1': 0.00357936857662071, 'lambda_l2': 0.00010601895065003074, 'num_leaves': 20, 'feature_fraction': 0.4001572685512002, 'bagging_fraction': 0.481550253391267, 'bagging_freq': 2, 'min_child_samples': 17, 'max_depth': 817}. Best is trial 205 with value: 0.8875892335648526.
[I 2022-06-21 16:32:27,027] Trial 473 finished with value: 0.5 and parameters: {'lambda_l1': 0.0007725371673255903, 'lambda_l2': 0.0005764124912455184, 'num_leaves': 49, 'feature_fraction': 0.43190127768888215, 'bagging_fraction': 0.464969078951

[I 2022-06-21 16:34:35,991] Trial 492 finished with value: 0.8725766492494436 and parameters: {'lambda_l1': 0.0004668619824448935, 'lambda_l2': 0.0008422837619629946, 'num_leaves': 45, 'feature_fraction': 0.42346452451672967, 'bagging_fraction': 0.4788908833376862, 'bagging_freq': 2, 'min_child_samples': 10, 'max_depth': 788}. Best is trial 205 with value: 0.8875892335648526.
[I 2022-06-21 16:34:42,420] Trial 493 finished with value: 0.8491881449089267 and parameters: {'lambda_l1': 0.0007867987936307244, 'lambda_l2': 0.00016323598668836035, 'num_leaves': 50, 'feature_fraction': 0.43315615910098987, 'bagging_fraction': 0.9353034383285065, 'bagging_freq': 1, 'min_child_samples': 39, 'max_depth': 929}. Best is trial 205 with value: 0.8875892335648526.
[I 2022-06-21 16:34:49,208] Trial 494 finished with value: 0.874788100057837 and parameters: {'lambda_l1': 0.00034288447318799785, 'lambda_l2': 0.00027239853192653307, 'num_leaves': 42, 'feature_fraction': 0.47168367792903776, 'bagging_fract

In [131]:
# Get best trial based on metric score
trial = study.best_trial

In [132]:
# Print params for best trial
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

    lambda_l1: 9.301631683130897e-05
    lambda_l2: 0.00042116251159990197
    num_leaves: 59
    feature_fraction: 0.4100780643597869
    bagging_fraction: 0.4704677676270611
    bagging_freq: 1
    min_child_samples: 15
    max_depth: 996


In [180]:
# Best Score from HP Opt
trial.values[0]

0.886714470556963

- Aumento de performance considerável em relação ao valor baseline para um LGBM

##### 4.2) PCA Dimension Reduction + Hyper Parameter Optimization with Optuna

In [181]:
# Define objectiva funtion to maximize metric
def objective(trial):
    
    # PCA Parameter Grid
    pca_param = {
        "n_components": trial.suggest_int("n_components", 5, 100)
    }
    # Hyper Parameter Grid
    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "n_estimators": trial.suggest_int("max_depth", 100, 1000)
    }
    # Create model pipeline with param trial
    clf = Pipeline([("pca", PCA(**pca_param)),
                    ("lgbm_clf", lgb.LGBMClassifier(random_state=SEED, **param))])
    # Get CV Metric we want to maximize
    balanced_accuracy = np.mean(cross_val_score(clf, X, y, scoring=metric_scorer, cv=rkf, n_jobs=-1))
    
    return balanced_accuracy

In [182]:
study = optuna.create_study(direction="maximize")

[I 2022-06-17 15:30:06,080] A new study created in memory with name: no-name-adfd880b-812e-41f4-828a-f1b968db75c6


In [ ]:
study.optimize(objective, n_trials=500)

In [26]:
# Get best trial based on metric score
trial = study.best_trial

In [27]:
# Print params for best trial
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

    n_components: 98
    lambda_l1: 4.997984938308335e-06
    lambda_l2: 9.51579276153768e-06
    num_leaves: 44
    feature_fraction: 0.5421454819677247
    bagging_fraction: 0.48235662397332973
    bagging_freq: 4
    min_child_samples: 8
    max_depth: 563


In [28]:
# Best Score from HP Opt
trial.values[0]

0.7134464057614213

- Utilizar redução de dimensionalidade via PCA piorou bastante a performance do modelo

##### 4.3) Features Scaler + Hyper Parameter Optimization with Optuna

In [29]:
# Define objectiva funtion to maximize metric
def objective(trial):
    
    # List scalers to chose from
    scalers = trial.suggest_categorical("scalers", ['minmax', 'standard', 'robust'])
    # Define scalers
    if scalers == "minmax":
        scaler = MinMaxScaler()
    elif scalers == "standard":
        scaler = StandardScaler()
    else:
        scaler = RobustScaler()
    # LGBM Hyper Parameter Grid
    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "n_estimators": trial.suggest_int("max_depth", 100, 1000)
    }
    # Create model pipeline with param trial
    clf = Pipeline([("scaler", scaler),
                    ("lgbm_clf", lgb.LGBMClassifier(random_state=SEED, **param))])
    # Get CV Metric we want to maximize
    balanced_accuracy = np.mean(cross_val_score(clf, X, y, scoring=metric_scorer, cv=rkf, n_jobs=-1))
    
    return balanced_accuracy

In [30]:
study = optuna.create_study(direction="maximize")

[I 2022-06-07 22:15:09,043] A new study created in memory with name: no-name-9d027bd4-11fc-4b2d-8846-da6f6ffb30f6


In [ ]:
study.optimize(objective, n_trials=500)

In [32]:
# Get best trial based on metric score
trial = study.best_trial

In [33]:
# Print params for best trial
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

    scalers: robust
    lambda_l1: 4.3223282482835764e-07
    lambda_l2: 3.3796186878776776e-05
    num_leaves: 188
    feature_fraction: 0.650978565866076
    bagging_fraction: 0.9212536490874126
    bagging_freq: 7
    min_child_samples: 36
    max_depth: 956


In [34]:
# Best Score from HP Opt
trial.values[0]

0.8521054248039541

- A adição de um estágio de feature scaling antes do treinamento parece ter ajudado o modelo

##### 4.4) Boruta Feature Selection + Features Scaler + Hyper Parameter Optimization with Optuna

In [275]:
# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=SEED, )

In [276]:
feat_selector.fit(np.array(X), y)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	635
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	635
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	635
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	635
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	635
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	635
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	635
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	50
Rejected: 	585
Iteration: 	9 / 100
Confirmed: 	10
Tentative: 	40
Rejected: 	585
Iteration: 	10 / 100
Confirmed: 	10
Tentative: 	40
Rejected: 	585
Iteration: 	11 / 100
Confirmed: 	10
Tentative: 	40
Rejected: 	585
Iteration: 	12 / 100
Confirmed: 	12
Tentative: 	32
Rejected: 	591
Iteration: 	13 / 100
Confirmed: 	12
Tentative: 	32
Rejected: 	591
Iteration: 	14 / 100
Confirmed: 	12
Tentative: 	32
Rejected: 	591
Iteration: 	15 / 100
Confirmed: 	12
Tentative: 	32
Rejected: 	591
Iteration: 	16 / 100
Confirmed: 	1

BorutaPy(estimator=RandomForestClassifier(class_weight='balanced', max_depth=5,
                                          n_estimators=181, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x166410F7740),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x166410F7740, verbose=2)

In [277]:
print("Number of selected features: ", feat_selector.n_features_)

Number of selected features:  27


In [278]:
# Best Features (accord to Boruta)
X.columns[feat_selector.support_]

Index(['Freq.1715.32907573994', 'Freq.1793.31292765446',
       'Freq.1984.28695988636', 'Freq.2032.95119529926',
       'Freq.2148.55916353654', 'Freq.2182.52259691583',
       'Freq.2186.33577081196', 'Freq.2242.07973381149',
       'Freq.2761.81291676166', 'Freq.3044.21704373186',
       'Freq.3077.77601350217', 'Freq.3414.71158220371',
       'Freq.3425.70211639867', 'Freq.3912.82100942603',
       'Freq.4266.3135397872', 'Freq.4282.69712175929',
       'Freq.4305.85988898402', 'Freq.4318.14922038936',
       'Freq.4395.12541812139', 'Freq.4773.1748593189',
       'Freq.4823.05474215093', 'Freq.5085.12753419191',
       'Freq.5224.39772946441', 'Freq.5433.51287445961',
       'Freq.6079.05181901815', 'Freq.7738.28945568542',
       'Freq.8943.76551923189'],
      dtype='object')

In [279]:
# Filter most importante features
X_transform = feat_selector.transform(np.array(X))

In [280]:
# Define objectiva funtion to maximize metric
def objective(trial):
    
    # List scalers to chose from
    scalers = trial.suggest_categorical("scalers", ['minmax', 'standard', 'robust'])
    # Define scalers
    if scalers == "minmax":
        scaler = MinMaxScaler()
    elif scalers == "standard":
        scaler = StandardScaler()
    else:
        scaler = RobustScaler()
    # LGBM Hyper Parameter Grid
    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "n_estimators": trial.suggest_int("max_depth", 100, 1000)
    }
    # Create model pipeline with param trial
    clf = Pipeline([("scaler", scaler),
                    ("lgbm_clf", lgb.LGBMClassifier(random_state=SEED, **param))])
    # Get CV Metric we want to maximize
    balanced_accuracy = np.mean(cross_val_score(clf, X_transform, y, scoring=metric_scorer, cv=rkf, n_jobs=-1))
    
    return balanced_accuracy

In [281]:
study = optuna.create_study(direction="maximize")

[I 2022-06-22 12:23:24,685] A new study created in memory with name: no-name-9537cfd1-f771-4be9-84f2-c5085aacaac4


In [282]:
study.optimize(objective, n_trials=500)

[I 2022-06-22 12:23:33,031] Trial 0 finished with value: 0.5 and parameters: {'scalers': 'minmax', 'lambda_l1': 0.00010650389196195333, 'lambda_l2': 0.5284911398502122, 'num_leaves': 91, 'feature_fraction': 0.487892320827369, 'bagging_fraction': 0.928213850523036, 'bagging_freq': 4, 'min_child_samples': 68, 'max_depth': 116}. Best is trial 0 with value: 0.5.
[I 2022-06-22 12:23:33,706] Trial 1 finished with value: 0.5 and parameters: {'scalers': 'standard', 'lambda_l1': 0.00024207495676854067, 'lambda_l2': 1.9141138837209378e-06, 'num_leaves': 171, 'feature_fraction': 0.45548449794622425, 'bagging_fraction': 0.4576570690947096, 'bagging_freq': 5, 'min_child_samples': 84, 'max_depth': 244}. Best is trial 0 with value: 0.5.
[I 2022-06-22 12:23:34,257] Trial 2 finished with value: 0.5 and parameters: {'scalers': 'minmax', 'lambda_l1': 0.00028470085166451916, 'lambda_l2': 1.227952984016988e-07, 'num_leaves': 99, 'feature_fraction': 0.9874417190432293, 'bagging_fraction': 0.6133499179616299

[I 2022-06-22 12:24:27,689] Trial 21 finished with value: 0.8592067886104263 and parameters: {'scalers': 'standard', 'lambda_l1': 2.4065039926722546e-05, 'lambda_l2': 1.570508061367828e-05, 'num_leaves': 202, 'feature_fraction': 0.9917645229352783, 'bagging_fraction': 0.6831882741343556, 'bagging_freq': 4, 'min_child_samples': 28, 'max_depth': 725}. Best is trial 5 with value: 0.8710019716910196.
[I 2022-06-22 12:24:32,871] Trial 22 finished with value: 0.8713779969815186 and parameters: {'scalers': 'standard', 'lambda_l1': 7.304353833037338e-06, 'lambda_l2': 6.291546369282907e-08, 'num_leaves': 222, 'feature_fraction': 0.9392423667211711, 'bagging_fraction': 0.6760225543292433, 'bagging_freq': 3, 'min_child_samples': 19, 'max_depth': 833}. Best is trial 22 with value: 0.8713779969815186.
[I 2022-06-22 12:24:38,923] Trial 23 finished with value: 0.8747459322583161 and parameters: {'scalers': 'standard', 'lambda_l1': 0.0011259192845105394, 'lambda_l2': 5.388835644428738e-08, 'num_leaves

[I 2022-06-22 12:26:09,156] Trial 42 finished with value: 0.8681394836673474 and parameters: {'scalers': 'minmax', 'lambda_l1': 0.00042898405555965034, 'lambda_l2': 3.0771362823084766e-08, 'num_leaves': 183, 'feature_fraction': 0.8792173213292823, 'bagging_fraction': 0.539665647382743, 'bagging_freq': 1, 'min_child_samples': 9, 'max_depth': 880}. Best is trial 31 with value: 0.8836487496206924.
[I 2022-06-22 12:26:14,973] Trial 43 finished with value: 0.8737881053629891 and parameters: {'scalers': 'minmax', 'lambda_l1': 0.0008016617685338747, 'lambda_l2': 5.705124892639596e-07, 'num_leaves': 156, 'feature_fraction': 0.9119443593512041, 'bagging_fraction': 0.5894490661510684, 'bagging_freq': 2, 'min_child_samples': 13, 'max_depth': 936}. Best is trial 31 with value: 0.8836487496206924.
[I 2022-06-22 12:26:20,083] Trial 44 finished with value: 0.8752180070531463 and parameters: {'scalers': 'minmax', 'lambda_l1': 0.0001956553712468332, 'lambda_l2': 1.9810297930850482e-07, 'num_leaves': 90

[I 2022-06-22 12:27:43,290] Trial 62 finished with value: 0.8708441697191697 and parameters: {'scalers': 'robust', 'lambda_l1': 0.004916108164899716, 'lambda_l2': 1.142078775121887e-07, 'num_leaves': 156, 'feature_fraction': 0.8708465328314912, 'bagging_fraction': 0.5868893451991998, 'bagging_freq': 1, 'min_child_samples': 19, 'max_depth': 921}. Best is trial 31 with value: 0.8836487496206924.
[I 2022-06-22 12:27:48,226] Trial 63 finished with value: 0.8688115109864334 and parameters: {'scalers': 'robust', 'lambda_l1': 0.05808091627402084, 'lambda_l2': 3.266790532452957e-07, 'num_leaves': 188, 'feature_fraction': 0.8451196666373337, 'bagging_fraction': 0.5452725959472474, 'bagging_freq': 1, 'min_child_samples': 10, 'max_depth': 838}. Best is trial 31 with value: 0.8836487496206924.
[I 2022-06-22 12:27:54,689] Trial 64 finished with value: 0.8684830714521119 and parameters: {'scalers': 'robust', 'lambda_l1': 0.001475220523645136, 'lambda_l2': 6.970202791100006e-07, 'num_leaves': 197, 'f

[I 2022-06-22 12:29:28,795] Trial 83 finished with value: 0.8788550160924388 and parameters: {'scalers': 'standard', 'lambda_l1': 0.0009984593970234243, 'lambda_l2': 7.591205265253566e-08, 'num_leaves': 162, 'feature_fraction': 0.8503663106899272, 'bagging_fraction': 0.5352040781193108, 'bagging_freq': 1, 'min_child_samples': 10, 'max_depth': 882}. Best is trial 31 with value: 0.8836487496206924.
[I 2022-06-22 12:29:34,579] Trial 84 finished with value: 0.8733723577334258 and parameters: {'scalers': 'standard', 'lambda_l1': 0.0005874617198536758, 'lambda_l2': 3.557389110981414e-07, 'num_leaves': 183, 'feature_fraction': 0.83975580450065, 'bagging_fraction': 0.5370577976793572, 'bagging_freq': 1, 'min_child_samples': 10, 'max_depth': 952}. Best is trial 31 with value: 0.8836487496206924.
[I 2022-06-22 12:29:40,240] Trial 85 finished with value: 0.8686329806674233 and parameters: {'scalers': 'standard', 'lambda_l1': 0.0010192521360826718, 'lambda_l2': 7.234580477109604e-07, 'num_leaves':

...

[I 2022-06-22 12:52:40,763] Trial 403 finished with value: 0.880622160197044 and parameters: {'scalers': 'minmax', 'lambda_l1': 6.039748814217228e-07, 'lambda_l2': 0.07081047323122279, 'num_leaves': 215, 'feature_fraction': 0.41523517302887114, 'bagging_fraction': 0.4336495409442528, 'bagging_freq': 2, 'min_child_samples': 7, 'max_depth': 715}. Best is trial 324 with value: 0.8860457100948587.
[I 2022-06-22 12:52:45,944] Trial 404 finished with value: 0.879525139162523 and parameters: {'scalers': 'minmax', 'lambda_l1': 0.001301324268739333, 'lambda_l2': 0.009998874000345616, 'num_leaves': 234, 'feature_fraction': 0.44195455407399337, 'bagging_fraction': 0.4003995434083204, 'bagging_freq': 3, 'min_child_samples': 13, 'max_depth': 934}. Best is trial 324 with value: 0.8860457100948587.
[I 2022-06-22 12:52:50,193] Trial 405 finished with value: 0.8756150638577108 and parameters: {'scalers': 'minmax', 'lambda_l1': 3.7081444034990286e-06, 'lambda_l2': 0.04333143125554678, 'num_leaves': 224,

[I 2022-06-22 12:54:06,510] Trial 423 finished with value: 0.8737359527727177 and parameters: {'scalers': 'minmax', 'lambda_l1': 1.0151184010932386e-05, 'lambda_l2': 0.0020143240895745293, 'num_leaves': 242, 'feature_fraction': 0.4100711149217103, 'bagging_fraction': 0.4217348873994826, 'bagging_freq': 3, 'min_child_samples': 10, 'max_depth': 610}. Best is trial 324 with value: 0.8860457100948587.
[I 2022-06-22 12:54:08,990] Trial 424 finished with value: 0.8667789924428856 and parameters: {'scalers': 'minmax', 'lambda_l1': 1.5954425977148137e-06, 'lambda_l2': 0.002871167731614152, 'num_leaves': 231, 'feature_fraction': 0.42195791797257687, 'bagging_fraction': 0.48990922269470116, 'bagging_freq': 3, 'min_child_samples': 26, 'max_depth': 518}. Best is trial 324 with value: 0.8860457100948587.
[I 2022-06-22 12:54:15,095] Trial 425 finished with value: 0.8738322260814521 and parameters: {'scalers': 'minmax', 'lambda_l1': 0.0016372010670453654, 'lambda_l2': 0.008563908632198216, 'num_leave

[I 2022-06-22 12:55:25,378] Trial 444 finished with value: 0.8817689606789838 and parameters: {'scalers': 'minmax', 'lambda_l1': 2.807377985791381e-06, 'lambda_l2': 1.3767166736272074e-08, 'num_leaves': 228, 'feature_fraction': 0.4454500640407263, 'bagging_fraction': 0.4382327166990512, 'bagging_freq': 3, 'min_child_samples': 5, 'max_depth': 632}. Best is trial 324 with value: 0.8860457100948587.
[I 2022-06-22 12:55:29,154] Trial 445 finished with value: 0.8767286242094678 and parameters: {'scalers': 'minmax', 'lambda_l1': 8.345354091838188e-06, 'lambda_l2': 2.71799957118158e-08, 'num_leaves': 234, 'feature_fraction': 0.4653156746873899, 'bagging_fraction': 0.6801092215665016, 'bagging_freq': 4, 'min_child_samples': 7, 'max_depth': 598}. Best is trial 324 with value: 0.8860457100948587.
[I 2022-06-22 12:55:33,946] Trial 446 finished with value: 0.8796720529212788 and parameters: {'scalers': 'minmax', 'lambda_l1': 4.479831307382189e-06, 'lambda_l2': 0.007429999779819605, 'num_leaves': 2

[I 2022-06-22 12:56:53,651] Trial 464 finished with value: 0.8780635506134732 and parameters: {'scalers': 'minmax', 'lambda_l1': 3.338745988830862e-05, 'lambda_l2': 0.0031307383510765204, 'num_leaves': 236, 'feature_fraction': 0.45255452765296345, 'bagging_fraction': 0.6677060648585293, 'bagging_freq': 3, 'min_child_samples': 15, 'max_depth': 707}. Best is trial 324 with value: 0.8860457100948587.
[I 2022-06-22 12:56:58,347] Trial 465 finished with value: 0.8807520680858528 and parameters: {'scalers': 'minmax', 'lambda_l1': 5.4079394366272204e-05, 'lambda_l2': 0.005045734507853339, 'num_leaves': 229, 'feature_fraction': 0.4204983486464603, 'bagging_fraction': 0.48148066837281733, 'bagging_freq': 3, 'min_child_samples': 12, 'max_depth': 737}. Best is trial 324 with value: 0.8860457100948587.
[I 2022-06-22 12:57:02,882] Trial 466 finished with value: 0.8711710729930546 and parameters: {'scalers': 'minmax', 'lambda_l1': 1.7868531329506287e-05, 'lambda_l2': 0.001926246682730383, 'num_leave

[I 2022-06-22 12:58:17,781] Trial 484 finished with value: 0.8770583290118894 and parameters: {'scalers': 'minmax', 'lambda_l1': 1.433626564102245e-05, 'lambda_l2': 0.003910273830979929, 'num_leaves': 248, 'feature_fraction': 0.41947518984840815, 'bagging_fraction': 0.43121228729720423, 'bagging_freq': 1, 'min_child_samples': 15, 'max_depth': 553}. Best is trial 324 with value: 0.8860457100948587.
[I 2022-06-22 12:58:22,021] Trial 485 finished with value: 0.8774274724243765 and parameters: {'scalers': 'minmax', 'lambda_l1': 1.885458127913055e-06, 'lambda_l2': 0.005236252891708598, 'num_leaves': 241, 'feature_fraction': 0.8173470733853108, 'bagging_fraction': 0.4490773992044536, 'bagging_freq': 3, 'min_child_samples': 12, 'max_depth': 609}. Best is trial 324 with value: 0.8860457100948587.
[I 2022-06-22 12:58:27,383] Trial 486 finished with value: 0.8722169933609559 and parameters: {'scalers': 'minmax', 'lambda_l1': 8.115259976526958e-06, 'lambda_l2': 0.005986203857225213, 'num_leaves':

In [284]:
# Get best trial based on metric score
trial = study.best_trial

In [285]:
# Print params for best trial
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

    scalers: minmax
    lambda_l1: 4.75261507145719e-06
    lambda_l2: 0.011742802261727603
    num_leaves: 241
    feature_fraction: 0.4109969027768354
    bagging_fraction: 0.4003975536156305
    bagging_freq: 3
    min_child_samples: 11
    max_depth: 655


In [286]:
# Best Score from HP Opt
trial.values[0]

0.8860457100948587

In [287]:
gbm_opt = lgb.LGBMClassifier(random_state=SEED, **trial.params)

- Pré selecionar as features mais importantes auxiliou a obter uma performance ainda melhor a partir do pipeline com Robust Scaler e LightGBM

In [288]:
X_transform_test = feat_selector.transform(np.array(X_test))

In [289]:
X_transform_test.shape

(64, 27)

In [290]:
gbm_opt.fit(X_transform,y)

[LightGBM] [Warning] Unknown parameter: scalers
[LightGBM] [Warning] feature_fraction is set=0.4109969027768354, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4109969027768354
[LightGBM] [Warning] lambda_l1 is set=4.75261507145719e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.75261507145719e-06
[LightGBM] [Warning] bagging_fraction is set=0.4003975536156305, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4003975536156305
[LightGBM] [Warning] lambda_l2 is set=0.011742802261727603, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.011742802261727603
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


LGBMClassifier(bagging_fraction=0.4003975536156305, bagging_freq=3,
               feature_fraction=0.4109969027768354,
               lambda_l1=4.75261507145719e-06, lambda_l2=0.011742802261727603,
               max_depth=655, min_child_samples=11, num_leaves=241,
               random_state=42, scalers='minmax')

In [291]:
y_pred = gbm_opt.predict(X_transform_test)

In [292]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        MILD       0.67      0.74      0.70        42
      SEVERE       0.39      0.32      0.35        22

    accuracy                           0.59        64
   macro avg       0.53      0.53      0.53        64
weighted avg       0.58      0.59      0.58        64



### Testing other algorithms 

In [233]:
X_test, y_test = test_mod.drop("Group", axis=1), test_mod["Group"]
X, y = train_mod.drop("Group", axis=1), train_mod["Group"]

###### Get most important features from boruta

In [235]:
# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=SEED, )
feat_selector.fit(np.array(X), y)
print("Number of selected features: ", feat_selector.n_features_)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	635
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	635
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	635
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	635
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	635
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	635
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	635
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	50
Rejected: 	585
Iteration: 	9 / 100
Confirmed: 	10
Tentative: 	40
Rejected: 	585
Iteration: 	10 / 100
Confirmed: 	10
Tentative: 	40
Rejected: 	585
Iteration: 	11 / 100
Confirmed: 	10
Tentative: 	40
Rejected: 	585
Iteration: 	12 / 100
Confirmed: 	12
Tentative: 	32
Rejected: 	591
Iteration: 	13 / 100
Confirmed: 	12
Tentative: 	32
Rejected: 	591
Iteration: 	14 / 100
Confirmed: 	12
Tentative: 	32
Rejected: 	591
Iteration: 	15 / 100
Confirmed: 	12
Tentative: 	32
Rejected: 	591
Iteration: 	16 / 100
Confirmed: 	1

In [236]:
# Filter most importante features
X_transform = feat_selector.transform(np.array(X))
X_transform_test = feat_selector.transform(np.array(X_test))

In [237]:
gbc = GradientBoostingClassifier(random_state = SEED)

In [238]:
parameters = {'max_depth':list(range(1,100,1)), 'subsample':list(np.arange(0.05,1,0.05)),
             'learning_rate':list(np.arange(0.05,1,0.05)), 'n_estimators':list(range(50,500,25)), }

In [239]:
rscv_gbc = RandomizedSearchCV(gbc, parameters, scoring=metric_scorer, n_jobs = -1, verbose = 4, n_iter = 200)

In [240]:
scores_gbc = rscv_gbc.fit(X,y)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [241]:
scores_gbc.best_score_

0.8336363636363636

In [245]:
y_pred = scores_gbc.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        MILD       0.65      0.57      0.61        42
      SEVERE       0.33      0.41      0.37        22

    accuracy                           0.52        64
   macro avg       0.49      0.49      0.49        64
weighted avg       0.54      0.52      0.53        64



In [246]:
scores_gbc_boruta = rscv_gbc.fit(X_transform, y)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [247]:
scores_gbc_boruta.best_score_

0.8705681818181817

In [248]:
scores_gbc_boruta.best_params_

{'subsample': 0.9000000000000001,
 'n_estimators': 200,
 'max_depth': 6,
 'learning_rate': 0.2}

In [250]:
y_pred = scores_gbc_boruta.predict(X_transform_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        MILD       0.63      0.74      0.68        42
      SEVERE       0.27      0.18      0.22        22

    accuracy                           0.55        64
   macro avg       0.45      0.46      0.45        64
weighted avg       0.51      0.55      0.52        64



In [251]:
lgbm = lgb.LGBMClassifier(random_state=SEED)
parameters = {'num_leaves':list(range(20,100,5)), 'min_child_samples':list(range(5,30,5)),'max_depth':list(range(-1,20,1)),
             'learning_rate':list(np.arange(0.05,1,0.05)),'reg_alpha':list(np.arange(0,0.6, 0.1))}
rscv_lgbm = RandomizedSearchCV(lgbm, parameters, scoring=metric_scorer, n_jobs = -1, verbose = 4, n_iter = 200)

In [259]:
scores_lgbm = rscv_lgbm.fit(X, y)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


,Freq.1198.05707939985,Freq.1204.07148226988,Freq.1211.25531677913,Freq.1217.79534957812,Freq.1223.24180676615,Freq.1234.42874922858,Freq.1239.04873179697,Freq.1243.98934968702,Freq.1249.10239635402,Freq.1254.7097579948,...,Freq.8812.07967315325,Freq.8943.76551923189,Freq.9058.85825530971,Freq.9098.58510797401,Freq.9437.74469644083,Freq.9593.90405666006,Freq.9799.842201746,Freq.10432.4853106264,Freq.11006.9514551194,Freq.11161.31855876
ID,,,,,,,,,,,,,,,,,,,,,
1,0.000319,0.000416,0.000077,4.034140e-05,0.000089,0.000282,0.000004,0.000130,0.000288,0.000000e+00,...,0.000004,0.000017,0.000005,0.000020,0.000029,0.000037,0.000024,0.000016,0.000030,3.036588e-05
10,0.000000,0.000000,0.000021,1.064014e-03,0.000113,0.001591,0.016310,0.002330,0.000476,6.074661e-04,...,0.000000,0.000040,0.000010,0.000001,0.000005,0.000002,0.000052,0.000003,0.000155,1.095279e-06
100,0.000199,0.000006,0.000013,5.089552e-08,0.000087,0.000543,0.000231,0.000068,0.000018,1.286428e-04,...,0.000020,0.000009,0.000044,0.000012,0.000067,0.000207,0.000020,0.000027,0.000100,1.003297e-04
101,0.000196,0.000244,0.000069,4.874671e-05,0.000125,0.000353,0.000029,0.000125,0.000120,4.988436e-07,...,0.000027,0.000072,0.000051,0.000013,0.000061,0.000040,0.000008,0.000003,0.000055,2.944864e-05
105,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000034,0.000027,0.000031,0.000014,0.000017,0.000028,0.000009,0.000012,0.000026,1.714090e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,0.000010,0.000394,0.000042,1.213255e-04,0.000052,0.000586,0.014572,0.001462,0.000112,2.465139e-04,...,0.000006,0.000003,0.000005,0.000007,0.000003,0.000011,0.000003,0.000016,0.001243,6.332741e-08
87,0.000048,0.000095,0.000039,7.052825e-05,0.000100,0.000121,0.001347,0.000230,0.000061,6.719760e-05,...,0.000000,0.000000,0.000023,0.000054,0.000042,0.000016,0.000023,0.000006,0.000498,0.000000e+00
93,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000004,0.000012,0.000016,0.000014,0.000062,0.000028,0.000025,0.000010,0.000178,9.164915e-05


In [253]:
scores_lgbm.best_score_

0.8252272727272727

In [261]:
X_test.shape

(64, 635)

In [263]:
y_pred = scores_lgbm.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        MILD       0.67      0.67      0.67        42
      SEVERE       0.36      0.36      0.36        22

    accuracy                           0.56        64
   macro avg       0.52      0.52      0.52        64
weighted avg       0.56      0.56      0.56        64



In [268]:
scores_lgbm_boruta = rscv_lgbm.fit(X_transform, y)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [274]:
scores_lgbm_boruta.best_score_

0.8743181818181818

In [270]:
X_transform_test.shape

(64, 27)

In [272]:
y_pred = scores_lgbm_boruta.predict(X_transform_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        MILD       0.62      0.60      0.61        42
      SEVERE       0.29      0.32      0.30        22

    accuracy                           0.50        64
   macro avg       0.46      0.46      0.46        64
weighted avg       0.51      0.50      0.50        64



In [ ]:
svc_poly = svc.LGBMClassifier(random_state=SEED)
parameters = {'num_leaves':list(range(20,100,5)), 'min_child_samples':list(range(5,30,5)),'max_depth':list(range(-1,20,1)),
             'learning_rate':list(np.arange(0.05,1,0.05)),'reg_alpha':list(np.arange(0,0.6, 0.1))}
rscv_lgbm = RandomizedSearchCV(lgbm, parameters, scoring=metric_scorer, n_jobs = -1, verbose = 4, n_iter = 200)